# Persona & Role‑Play Workbench  
Interactively adjust persona tone, reading level, and length limits before running a simulated dialogue.

In [ ]:
!pip -q install openai ipywidgets textstat

In [ ]:
import os, json, openai, ipywidgets as w, textstat
from IPython.display import display, Markdown, clear_output

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'sk-')

# --- Widgets ---
scenario_box = w.Textarea(
    value="Topic: Explaining healthy sleep habits",
    description="Scenario:",
    layout=w.Layout(width='100%', height='60px'))

n_roles = w.IntSlider(value=2, min=1, max=5, step=1, description='Roles')
max_tokens = w.IntSlider(value=400, min=50, max=800, step=50, description='Max tokens')

# dynamic persona widgets
role_widgets = []

def rebuild_roles(change=None):
    global role_widgets
    for rw in role_widgets:
        rw['box'].close()
    role_widgets = []
    for i in range(n_roles.value):
        name = w.Text(value=f'Person{i+1}', description='Name')
        tone = w.Dropdown(options=['formal','casual','friendly','professional','sarcastic'],
                          value='formal', description='Tone')
        reading = w.IntSlider(value=10, min=1, max=12, description='ReadLvl')
        length = w.IntSlider(value=50, min=5, max=200, description='Length')
        box = w.VBox([w.HTML(f'<b>Role {i+1}</b>'), name, tone, reading, length])
        role_widgets.append({'name': name, 'tone': tone, 'reading_level': reading,
                             'length': length, 'box': box})
    roles_vbox.children = [rw['box'] for rw in role_widgets]

roles_vbox = w.VBox()
n_roles.observe(rebuild_roles, names='value')
rebuild_roles()

run_btn = w.Button(description='Simulate 👉')
out = w.Output()

def parse_roles():
    roles = []
    for rw in role_widgets:
        roles.append({
            'name': rw['name'].value,
            'tone': rw['tone'].value,
            'reading_level': rw['reading_level'].value,
            'length': rw['length'].value
        })
    return roles

def build_prompt(roles, scenario):
    sys_msg = "You are participating in a role‑play conversation. Follow persona definitions strictly."
    msgs = [{'role':'system','content': sys_msg},
            {'role':'system','content': f"Scenario: {scenario}"}]
    # prime each persona
    for r in roles:
        msgs.append({'role':'system','content':
                     f"Persona {r.get('name','')} — Tone: {r.get('tone','')}; "
                     f"Reading_Level: {r.get('reading_level','')}; Length limit: <= {r.get('length','')} words"})
    # kick‑off
    msgs.append({'role':'user','content': "Begin the conversation."})
    return msgs

def run_sim(b):
    with out:
        out.clear_output()
        roles = parse_roles()
        msgs = build_prompt(roles, scenario_box.value)
        print("Prompt:", json.dumps(msgs, indent=2))
        try:
            resp = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=msgs,
                temperature=0.7,
                max_tokens=max_tokens.value
            )
            text = resp.choices[0].message.content
            display(Markdown("### 🗣️ Conversation
" + text))
            grade = textstat.flesch_kincaid_grade(text)
            print(f"Flesch‑Kincaid Grade Level ≈ {grade:.1f}")
        except Exception as e:
            print("Error:", e)

run_btn.on_click(run_sim)

ui = w.VBox([scenario_box, n_roles, max_tokens, roles_vbox, run_btn, out])
display(ui)
print("Define personas, adjust settings, then **Simulate**!")



---  
### Tips  
* Use explicit delimiters like `<<Patient>>` / `<<Doctor>>` inside personas.  
* Combine with length limits for concise turn‑taking.  
* Evaluate reading level to match your audience.